In [3]:
from typing import Annotated
from dotenv import load_dotenv
import os

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI  # Correct Gemini wrapper

# Load API key from .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# Initialize Gemini LLM with API key
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=api_key
)


In [11]:
# Define state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Chatbot node (correct version)
def chatbot(state: State):
    ai_message = llm.invoke(state["messages"])
    return {"messages": [ai_message]}  # IMPORTANT: wrap in list


# Build graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [13]:
# Multi-turn interactive loop
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break

    if state is None:
        state = {"messages": [{"role": "user", "content": in_message}]}
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("You:", in_message)
    print("Bot:", state["messages"][-1].content)


You: who was first man on moon? name only
Bot: Neil Armstrong
You: what year?
Bot: 1969
You: who was with him?
Bot: Buzz Aldrin


In [14]:
print(graph.get_graph().draw_mermaid())


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot_node(chatbot_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot_node;
	chatbot_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

